In [1]:
from gpu_rocket import torchrocket
import numpy as np
import torch

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
num_examples = 3
num_channels = 20
timesteps = 100

cuda:0


In [3]:
# either: 
# -use the kernels from sktime (and then convert to 32bit if you want 32bit gpu kernels)
# -generate them yourself
# -or use the one I have edited from sktime to make 32bit 

kernels = torchrocket._generate_kernels_sktime_32bit_version(timesteps, 10_000, num_channels, seed=None)
# Note: it is required to convert each of these np arrays to lists if you want to script the model (script doesnt work with np)
# e.g. channel_indices = channel_indices.tolist() etc

In [4]:
X = np.random.rand(num_examples,num_channels,timesteps).astype(np.float32)
X_torch =torch.from_numpy(X.astype(np.float32)).to(device)

In [5]:
torch_rocket_softPPV_10k = torchrocket.TorchRocket(kernels,ppv_ver='softPPV',softppv_param=10_000.)
torch_rocket_softPPV_10k.to(device)
torch_rocket_softPPV_10k.eval()
torch_rocket_softPPV_5 = torchrocket.TorchRocket(kernels,ppv_ver='softPPV',softppv_param=5.)
torch_rocket_softPPV_5.to(device)
torch_rocket_softPPV_5.eval()
torch_rocket_heaviside_PPV = torchrocket.TorchRocket(kernels,ppv_ver='heaviside_PPV')
torch_rocket_heaviside_PPV.to(device)
torch_rocket_heaviside_PPV.eval()
print(f'')

In [6]:
with torch.no_grad():
    out_softPPV_10k = torch_rocket_softPPV_10k(X_torch)
    out_softPPV_5 = torch_rocket_softPPV_5(X_torch)
    out_heaviside_PPV = torch_rocket_heaviside_PPV(X_torch)

In [7]:
# compare to sktime implementation
from sktime.transformations.panel.rocket import _rocket
# Note: sktime rocket uses 64bit floats (we changed the sktime code to use 32bit for our timings)
(weights,lengths,biases,dilations,paddings,
        num_channel_indices,channel_indices) = kernels
weights = weights.astype(np.float64)
biases = biases.astype(np.float64)

kernels_64 = (weights,lengths,biases,dilations,paddings,
    num_channel_indices,channel_indices)

out_sktime_rocket = _rocket._apply_kernels(X.astype(np.float64), kernels_64)

In [8]:
print(f'out_softPPV shape: {out_softPPV_10k.shape}')
print(f'out_softPPV shape: {out_softPPV_5.shape}')
print(f'out_heaviside_PPV shape: {out_heaviside_PPV.shape}')
print(f'out_sktime_rocket shape: {out_sktime_rocket.shape}')

out_softPPV shape: torch.Size([3, 20000])
out_softPPV shape: torch.Size([3, 20000])
out_heaviside_PPV shape: torch.Size([3, 20000])
out_sktime_rocket shape: (3, 20000)


In [9]:
for i in range(out_softPPV_5.shape[1]):
    print(f'{i}: {out_softPPV_5[0][i]},{out_softPPV_10k[0][i]},{out_heaviside_PPV[0][i]},{out_sktime_rocket[0][i]}')

0: 0.11845286190509796,0.23749999701976776,0.23749999701976776,0.2375
1: 1.8178653717041016,1.8178653717041016,1.8178653717041016,1.8178655496627032
2: 0.46726804971694946,0.6299999952316284,0.6299999952316284,0.63
3: 4.161397457122803,4.161397457122803,4.161397457122803,4.1613968389126015
4: 0.4537818431854248,0.5799999833106995,0.5799999833106995,0.58
5: 3.901569366455078,3.901569366455078,3.901569366455078,3.901569023994438
6: 0.35089772939682007,0.46666669845581055,0.46666669845581055,0.4666666666666667
7: 4.472898960113525,4.472898960113525,4.472898960113525,4.472899457682504
8: 0.14486390352249146,0.23529411852359772,0.23529411852359772,0.23529411764705882
9: 1.2803531885147095,1.2803531885147095,1.2803531885147095,1.2803530671408505
10: 0.589854896068573,0.699999988079071,0.699999988079071,0.7
11: 4.594573974609375,4.594573974609375,4.594573974609375,4.594574552967862
12: 0.381447970867157,0.47826087474823,0.47826087474823,0.4782608695652174
13: 4.044003486633301,4.0440034866333